In [1]:
# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")

In [2]:
# import the necessary packages
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import img_to_array
from keras.utils import to_categorical
#from pyimagesearch.lenet import LeNet
from imutils import paths

Using TensorFlow backend.


In [3]:
# import the necessary packages
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.layers.core import Dropout
from keras import backend as K

In [4]:
class LeNet:
	@staticmethod
	def build(width, height, depth, classes):
		# initialize the model
		model = Sequential()
		inputShape = (height, width, depth)

		# if we are using "channels first", update the input shape
		if K.image_data_format() == "channels_first":
			inputShape = (depth, height, width)

		# first set of CONV => RELU => POOL layers
		model.add(Conv2D(64, (3, 3), padding="same",
			input_shape=inputShape))
		model.add(Activation("relu"))
		model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

		# second set of CONV => RELU => POOL layers
		model.add(Conv2D(128, (3, 3), padding="same"))
		model.add(Activation("relu"))
		model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
        
		# third set of CONV => RELU => POOL layers
		model.add(Conv2D(256, (3, 3), padding="same"))
		model.add(Activation("relu"))
		model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
        
		# third set of CONV => RELU => POOL layers
		model.add(Conv2D(512, (3, 3), padding="same"))
		model.add(Activation("relu"))
		model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
              

		# first (and only) set of FC => RELU layers
		model.add(Flatten())
		model.add(Dense(1000))
		model.add(Activation("relu"))
		model.add(Dropout(0.5))

		# softmax classifier
		model.add(Dense(classes))
		model.add(Activation("softmax"))

		# return the constructed network architecture
		return model

In [5]:
import matplotlib.pyplot as plt
import numpy as np
import argparse
import random
import csv
import pandas as pd
import re
import cv2
import os

In [6]:
# This is code from the original tutorial for parsing the images
# construct the argument parse and parse the args
#ap = argparse.ArgumentParser()
#ap.add_argument("-d", "--dataset", required =True,
#               help="path to input dataset")
#ap.add_argument("-m", "--model", required=True,
#               help="path to output model")
#ap.add_argument("-p", "--plot", type=str, default="plot.png",
#               help="path to output accuracy/loss plot")
#args = vars(ap.parse_args())

In [14]:
# init the number of epochs to train for, init learning rate and batch size
EPOCHS = 20
INIT_LR = 1e-3
BS = 32

In [8]:
# init the image suffix, yset, and image list
print("[INFO] loading images...")
suffix = '.jpg'
img_list = []
yset = []

[INFO] loading images...


In [9]:
# create labels list and 2 dicts for 2 way mapping
labels = []
# key = label value = number
label_yval = dict()
# key = number value = label
yval_label = dict()

In [10]:
# use csv file to grab images/labels
df = pd.read_csv('zaslavsk_Cyclops_Cave_Ceramic_Petrography.csv')
nameCol = df['#img']
fabricCol = df['Fabric Code']

In [11]:
# add all fabric columns to the y set
for i in range (0,len(fabricCol)):
    labels.append(fabricCol[i])

In [12]:
# grab all unique labels
uni_labels = set(labels)
uni_labels = list(uni_labels)

In [13]:
# assign each label a dict key number
for i in range(0,len(uni_labels)):
    yval_label[i] = uni_labels[i]
    label_yval[uni_labels[i]] = i

In [15]:
label_yval

{'Cyclops Cave 1': 1,
 'Cyclops Cave 10': 5,
 'Cyclops Cave 11': 0,
 'Cyclops Cave 12': 9,
 'Cyclops Cave 2': 3,
 'Cyclops Cave 3': 10,
 'Cyclops Cave 4': 4,
 'Cyclops Cave 5': 8,
 'Cyclops Cave 6': 2,
 'Cyclops Cave 7': 6,
 'Cyclops Cave 8': 11,
 'Cyclops Cave 9': 7}

In [16]:
len(labels)

252

In [17]:
label_yval[labels[0]]

1

In [18]:
# create list of keys associated with their labels
for i in range (0, len(labels)):
    yset.append(label_yval[labels[i]])

In [19]:
len(yset)

252

In [20]:
# remove text and leave fabric cave number for labels and zero index
#for i in range (0,len(yset)):
#    yset[i] = int(re.sub("\D", "", yset[i]))
#    yset[i] = yset[i]-1

In [21]:
# gather images from path created from file names in csv file
for i in range (0,len(nameCol)):
    base_filename = nameCol[i]
    fileName = os.path.join("./Cyclops Cave/images/", base_filename + suffix)
    im = cv2.imread(fileName)
    im = cv2.resize(im, (28,28))
    im = img_to_array(im)
    img_list.append(im)

In [22]:
# split the test and training set 75:25
split = int(len(img_list)*(.75))
xtrain = img_list[:split]
xtest = img_list[split:]
ytrain = yset[:split]
ytest = yset[split:]

In [23]:
# transform to arrays
trainX = np.array(xtrain, dtype="float")/225.0
testX = np.array(xtest, dtype ="float")/225.0

ytrain = np.array(ytrain)
ytest = np.array(ytest)

In [24]:
trainX.shape

(189, 28, 28, 3)

In [25]:
# parsed Y data containers
trainY = []
testY = []

In [26]:
# convert labels from int to vectors
trainY = np_utils.to_categorical(ytrain,12)
testY = np_utils.to_categorical(ytest,12)

In [27]:
# construct the image generator for data augmentation
aug = ImageDataGenerator(rotation_range=30, width_shift_range=0.1,
                        height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
                        horizontal_flip=True, fill_mode="nearest")

In [28]:
# initialize the model
print("[INFO] compiling model...")
model = LeNet.build(width=28, height=28, depth=3, classes=12)
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="categorical_crossentropy", optimizer=opt,
                metrics=["accuracy"])

[INFO] compiling model...


In [29]:
# train the network
print("[INFO] training network...")
H = model.fit_generator(aug.flow(trainX, trainY, batch_size=BS),
    validation_data=(testX, testY), steps_per_epoch=len(trainX) // BS,
    epochs=EPOCHS, verbose=1)

[INFO] training network...
Epoch 1/20
6/5 [====================================] - 6s 943ms/step - loss: 2.1904 - acc: 0.4426 - val_loss: 2.6768 - val_acc: 0.3016
Epoch 2/20
6/5 [====================================] - 4s 700ms/step - loss: 1.3890 - acc: 0.6640 - val_loss: 3.4917 - val_acc: 0.3016
Epoch 3/20
6/5 [====================================] - 4s 698ms/step - loss: 1.2679 - acc: 0.6623 - val_loss: 4.6158 - val_acc: 0.3016
Epoch 4/20
6/5 [====================================] - 4s 694ms/step - loss: 1.2787 - acc: 0.6589 - val_loss: 3.9602 - val_acc: 0.3016
Epoch 5/20
6/5 [====================================] - 4s 700ms/step - loss: 1.1907 - acc: 0.6703 - val_loss: 4.5723 - val_acc: 0.3016
Epoch 6/20
6/5 [====================================] - 4s 681ms/step - loss: 1.1767 - acc: 0.6534 - val_loss: 4.3721 - val_acc: 0.3016
Epoch 7/20
6/5 [====================================] - 4s 706ms/step - loss: 1.0983 - acc: 0.6789 - val_loss: 5.2730 - val_acc: 0.3016
Epoch 8/20
6/5 [=====

In [30]:
# plot the training loss and accuracy 
plt.style.use("ggplot")
plt.figure()
N = EPOCHS
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["acc"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_acc"], label="val_acc")
plt.title("Training Loss and Accuracy on Label Prediction")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig('figure.png')

In [31]:
chart = cv2.imread('figure.png',1)
cv2.imshow('Results',chart)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [32]:
img_check = np.array(img_list, dtype ="float")/225.0

In [33]:
trainX.shape

(189, 28, 28, 3)

In [35]:
predictionsMade = []

In [41]:
len(img_check)

252

In [45]:
for i in range (0,len(img_check)):
    print(i)
    preds = model.predict(img_check)[i]
    predIndex = np.where(preds == np.amax(preds))
    prediction = int(predIndex[0][0])
    predictionsMade.append(prediction)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251


In [43]:
preds = model.predict(img_check)[100]

In [150]:
preds

array([  5.36204534e-05,   9.40667242e-02,   9.24172066e-03,
         4.77579197e-05,   2.65737344e-03,   4.70681377e-02,
         6.55007809e-02,   2.08184216e-02,   6.87556021e-05,
         7.58650839e-01,   1.75984204e-03,   6.60259611e-05], dtype=float32)

In [151]:
predIndex = np.where(preds == np.amax(preds))

In [152]:
predIndex

(array([9], dtype=int64),)

In [153]:
prediction = int(predIndex[0][0])

In [46]:
predictionsMade

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 11,
 1,
 11,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 11,
 11,
 11,
 11,
 11,
 1,
 1,
 1,
 1,
 11,
 1,
 11,
 1,
 1,
 1,
 1,
 11,
 11,
 1,
 11,
 1,
 1,
 1,
 1,
 1,
 11,
 1,
 11,
 1,
 11,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 11,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 11,
 1,
 11,
 1,
 11,
 1,
 11,
 1,
 1,
 1,
 11,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 11,
 1,
 11,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 11,
 11,
 11,
 11,
 11,
 1,
 1,
 1,
 1,
 11,
 1,
 11,
 1,
 1,
 1,
 1,
 11,
 11,
 1,
 11,
 1,
 1,
 1,
 1,
 1,
 11,
 1,
 11,
 1,
 11,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 11,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 11,
 1,
 11,
 

In [155]:
type(prediction)

int

In [156]:
prediction

9

In [158]:
if(prediction == yset[100]):
    print("yes")

In [70]:
trainY[12]

array([ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.])

In [81]:
yset[2]

9

In [ ]:
!pip 